In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'tpu-getting-started:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F21154%2F1243559%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240207%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240207T130418Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5b55e81782654340fcb02731055400e18934a2cbeb694866fe80e47c6c020c9b3f7ae0fb9973cffea9a148817ea4287b18789f21345ea496221c801b649b7c868c7edaad2d3195213614d219c057133c0f4cb0baca4d17866cce929d6995b8c8b9ace8bd27549afe3c431e0710f3939403e6a940728fc335c891cf2d1a7e4d3ef426ec6a81e1d66ace3815dfa4c98c7b2821112190a6c9ab013d64433c64f20684101f4530478a5a11a7dcc6fa14f78f7796a7fbd4326ae2a015ce7cc3f6f915b37a215d2c4c55ac984944026698d80f9030af2bc4afaa8050fb45232f932e9d0a14c0f37ccacda6f4d1fd3dfe5f16dd57064b8f64a9157a426856954b82f4bc'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Подготовка

## Импорты

In [ ]:
!pip install -q efficientnet

In [ ]:
# from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
# from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.applications.xception import Xception
# from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2
# from tensorflow.keras.applications.nasnet import NASNetLarge
from efficientnet.tfkeras import EfficientNetB7, EfficientNetL2, EfficientNetB0, EfficientNetB1

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dropout, BatchNormalization, GaussianDropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
#  библиотека для работы с наборами данных на Kaggle
from kaggle_datasets import KaggleDatasets
import re
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
print("Tensorflow version " + tf.__version__)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Обнаружение оборудования, возврат соответствующей стратегии распространения: TPU, GPU, CPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Обнаружение TPU. Параметры среды не требуются, если задана переменная среды TPU_NAME. На Kaggle это всегда так.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # стратегия распространения по умолчанию в Tensorflow. Работает на CPU и одном GPU.
print("REPLICAS: ", strategy.num_replicas_in_sync)

## Данные

In [ ]:
# GCS_DS_PATH = 'gs://kds-dd5b7308dd0932436652aceb10f871ab80411ed4a570d2878ac7e9bf'

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path()
GCS_DS_PATH#получаем путь к наборам данных

In [ ]:
IMAGE_SIZE = [512, 512] # при таком размере графическому процессору не хватит памяти. Используйте TPU
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

GCS_PATH_SELECT = { # available image sizes
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192',
    224: GCS_DS_PATH + '/tfrecords-jpeg-224x224',
    331: GCS_DS_PATH + '/tfrecords-jpeg-331x331',
    512: GCS_DS_PATH + '/tfrecords-jpeg-512x512'
}
GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

SEED = 2020

## Функции

In [ ]:
def decode_image(image_data):
    """Декодирует изображение в vyjujvthye. vfnhbwe (тензор)
    Нормализует данные и преобразовывает изображения к указанному размеру"""
    image = tf.image.decode_jpeg(image_data, channels=3) # Декодирование изображения в формате JPEG в тензор uint8.
    image = tf.cast(image, tf.float32) / 255.0  # преобразовать изображение в плавающее в диапазоне [0, 1]
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # явный размер, необходимый для TPU
#     image = tf.keras.applications.inception_resnet_v2.preprocess_input(image)
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string означает байтовую строку
        "class": tf.io.FixedLenFeature([], tf.int64),  # [] означает отдельный элемент
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT) # парсим отдельный пример в указанном формате
    image = decode_image(example['image']) # преобразуем изображение к нужному нам формату
    label = tf.cast(example['class'], tf.int32)
    return image, label # возвращает набор данных пар (изображение, метка)

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string означает байтовую строку
        "id": tf.io.FixedLenFeature([], tf.string),  # [] означает отдельный элемент
        # класс отсутствует, задача этого конкурса - предсказать классы цветов для тестового набора данных
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image']) # преобразуем изображение к нужному нам формату
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    """Читает из TFRecords. Для оптимальной производительности одновременное чтение из нескольких
    файлов без учета порядка данных. Порядок не имеет значения, поскольку мы все равно будем перетасовывать данные"""

    ignore_order = tf.data.Options() # Представляет параметры для tf.data.Dataset.
    if not ordered:
        ignore_order.experimental_deterministic = False # отключить порядок, увеличить скорость

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # автоматически чередует чтение из нескольких файлов
    dataset = dataset.with_options(ignore_order) # использует данные сразу после их поступления, а не в исходном порядке
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # возвращает набор данных пар (изображение, метка), если метка = Истина, или пар (изображение, идентификатор), если метка = Ложь
    return dataset

def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    flag = random.randint(1,3)
    # coef_1 = random.randint(70, 90) * 0.01
    # coef_2 = random.randint(70, 90) * 0.01
    if flag == 1:
        image = tf.image.random_flip_left_right(image, seed=SEED)
    elif flag == 2:
        image = tf.image.random_flip_up_down(image, seed=SEED)
    # else:
        # image = tf.image.random_crop(image, [int(IMAGE_SIZE[0]*coef_1), int(IMAGE_SIZE[0]*coef_2), 3],seed=SEED, )
    return image, label

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # набор обучающих данных должен повторяться в течение нескольких эпох
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) #готовим следующий набор, пока предыдущий обучается
    return dataset

def get_validation_dataset():
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=False)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache() # кешируем набор
    dataset = dataset.prefetch(AUTO) #готовим следующий набор, пока предыдущий обучается
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) #готовим следующий набор, пока предыдущий обучается
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

In [ ]:
def get_model(use_model, weights):
    # noisy-student
    base_model = use_model(weights=weights,
                      include_top=False, pooling='avg',
                      input_shape=(*IMAGE_SIZE, 3))
#     base_model.trainable = False
    x = base_model.output
    predictions = Dense(104, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=predictions)

In [ ]:
def print_score(history):

  plt.plot(history.history['sparse_categorical_accuracy'],
          label='Оценка точности на обучающем наборе')
  plt.plot(history.history['val_sparse_categorical_accuracy'],
          label='Оценка точности на проверочном наборе')
  plt.xlabel('Эпоха обучения')
  plt.ylabel('Оценка точности')
  plt.legend()
  plt.show()

def print_loss(history):

  plt.plot(history.history['loss'],
         label='Оценка потерь на обучающем наборе')
  plt.plot(history.history['val_loss'],
          label='Оценка потерь на проверочном наборе')
  plt.xlabel('Эпоха обучения')
  plt.ylabel('Оценка потерь')
  plt.legend()
  plt.show()


## Датасеты

In [ ]:
training_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()

# VGG19

In [ ]:
with strategy.scope():
    vgg_model = get_model(VGG19, 'imagenet') # тут подставить свою модель

vgg_model.compile(
    optimizer='nadam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [ ]:
VGG_EPOCHS = 30

In [ ]:
LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync#0.0001
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 10
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .75

def vgg_lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

vgg_lr_callback = tf.keras.callbacks.LearningRateScheduler(vgg_lrfn, verbose=True)

# построим график изменения шага обучение в зависимости от эпох
rng = [i for i in range(VGG_EPOCHS)]
y = [vgg_lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
history_vgg = vgg_model.fit(training_dataset,
          steps_per_epoch=STEPS_PER_EPOCH,
          epochs=VGG_EPOCHS,
          callbacks=[vgg_lr_callback, ModelCheckpoint(filepath='VGG19.h5',
                                                      monitor='val_loss',
                                                      save_best_only=True)],
          validation_data=validation_dataset,
          workers = 3)

In [ ]:
print_score(history_vgg)
print_loss(history_vgg)

# InceptionResNetV2

In [ ]:
with strategy.scope():
    inception_model = get_model(InceptionResNetV2, 'imagenet') # тут подставить свою модель

inception_model.compile(
    optimizer='nadam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [ ]:
INCEPTION_EPOCHS = 15

In [ ]:
LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync#0.0001
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 8
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .7

def inception_lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

inception_lr_callback = tf.keras.callbacks.LearningRateScheduler(inception_lrfn, verbose=True)

# построим график изменения шага обучение в зависимости от эпох
rng = [i for i in range(INCEPTION_EPOCHS)]
y = [inception_lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
history_inception = inception_model.fit(training_dataset,
          steps_per_epoch=STEPS_PER_EPOCH,
          epochs=INCEPTION_EPOCHS,
          callbacks=[inception_lr_callback, ModelCheckpoint(filepath='INCEPTION.h5',
                                                            monitor='val_loss',
                                                            save_best_only=True)],
          validation_data=validation_dataset,
          workers = 3)

In [ ]:
print_score(history_inception)
print_loss(history_inception)

# Xception

In [ ]:
with strategy.scope():
    xception_model = get_model(Xception, 'imagenet') # тут подставить свою модель

xception_model.compile(
    optimizer='nadam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [ ]:
XCEPTION_EPOCHS = 15

In [ ]:
LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync#0.0001
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 8
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .7

def xception_lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

xception_lr_callback = tf.keras.callbacks.LearningRateScheduler(xception_lrfn, verbose=True)

# построим график изменения шага обучение в зависимости от эпох
rng = [i for i in range(XCEPTION_EPOCHS)]
y = [xception_lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
history_xception = xception_model.fit(training_dataset,
          steps_per_epoch=STEPS_PER_EPOCH,
          epochs=XCEPTION_EPOCHS,
          callbacks=[xception_lr_callback, ModelCheckpoint(filepath='XCEPTION.h5',
                                                            monitor='val_loss',
                                                            save_best_only=True)],
          validation_data=validation_dataset,
          workers = 3)

In [ ]:
print_score(history_xception)
print_loss(history_xception)

# ResNet152V2

In [ ]:
with strategy.scope():
    resnet_model = get_model(ResNet152V2, 'imagenet') # тут подставить свою модель

resnet_model.compile(
    optimizer='nadam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [ ]:
RESNET_EPOCHS = 15

In [ ]:
LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync#0.0001
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 8
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .7

def resnet_lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

resnet_lr_callback = tf.keras.callbacks.LearningRateScheduler(resnet_lrfn, verbose=True)

# построим график изменения шага обучение в зависимости от эпох
rng = [i for i in range(RESNET_EPOCHS)]
y = [resnet_lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
history_resnet = resnet_model.fit(training_dataset,
          steps_per_epoch=STEPS_PER_EPOCH,
          epochs=RESNET_EPOCHS,
          callbacks=[resnet_lr_callback, ModelCheckpoint(filepath='RESNET.h5',
                                                            monitor='val_loss',
                                                            save_best_only=True)],
          validation_data=validation_dataset,
          workers = 3)

In [ ]:
print_score(history_resnet)
print_loss(history_resnet)

# EfficientNetB7

In [ ]:
with strategy.scope():
    efficientnet_model = get_model(EfficientNetB7, 'noisy-student') # тут подставить свою модель

efficientnet_model.compile(
    optimizer='nadam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [ ]:
EFFICIENTNET_EPOCHS = 15

In [ ]:
LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync#0.0001
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 8
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .7

def efficientnet_lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

efficientnet_lr_callback = tf.keras.callbacks.LearningRateScheduler(efficientnet_lrfn, verbose=True)

# построим график изменения шага обучение в зависимости от эпох
rng = [i for i in range(EFFICIENTNET_EPOCHS)]
y = [efficientnet_lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
history_efficientnet = efficientnet_model.fit(training_dataset,
          steps_per_epoch=STEPS_PER_EPOCH,
          epochs=EFFICIENTNET_EPOCHS,
          callbacks=[efficientnet_lr_callback, ModelCheckpoint(filepath='EFFICIENTNET.h5',
                                                            monitor='val_loss',
                                                            save_best_only=True)],
          validation_data=validation_dataset,
          workers = 3)

In [ ]:
print_score(history_efficientnet)
print_loss(history_efficientnet)

# Тест на submission

In [ ]:
test_ds = get_test_dataset(ordered=True)

print('Вычисляем предсказания...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = efficientnet_model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

print('Создание файла submission.csv...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # все в одной партии
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')